In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import plotly.express as px
import plotly.graph_objects as go
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error
from tqdm import tqdm
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
from tensorflow.keras import Input, Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
import joblib
import plotly.io as pio

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
data_path = '/content/drive/MyDrive/BKW/LNG Project data/' # if running this notebook in colab
# data_path = '../../data/LNG Project data/' # if running this notebook locally

In [4]:
file_path = 'clean_data/Prices_since_2009_merged_Brent_Coal_Vol_LNG.csv'

In [5]:
# Load and prepare data
df_all = pd.read_csv(data_path + file_path)
df_all['Date'] = pd.to_datetime(df_all['Date'])
df_all.head()


,Date,Brent_Vol.,Brent_Price,CBOE_Vol.,CBOE_Price,Coal_Vol.,Coal_Price,HenryHub_Price,JKM_Vol.,JKM_Price,NBP_Price,PSV_Price,TTF_Price
0,2009-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.00
1,2009-01-02,85.07K,46.91,NaN,39.19,NaN,74.35,5.41,NaN,NaN,NaN,NaN,22.20
2,2009-01-05,109.47K,49.62,NaN,39.08,NaN,77.65,5.83,NaN,NaN,NaN,NaN,22.85
3,2009-01-06,119.83K,50.53,NaN,38.56,NaN,81.25,6.10,NaN,NaN,NaN,NaN,26.00
4,2009-01-07,130.44K,45.86,NaN,43.39,NaN,78.65,5.89,NaN,NaN,NaN,NaN,26.33


In [ ]:
df_all.describe()

,Date,Brent_Price,CBOE_Vol.,CBOE_Price,Coal_Price,HenryHub_Price,JKM_Price,NBP_Price,PSV_Price,TTF_Price
count,4307,4265.00000,0.0,4189.000000,4233.000000,4162.000000,2802.000000,4088.000000,3442.000000,4246.000000
mean,2017-04-04 19:51:35.147434496,77.17793,NaN,19.243275,112.900449,3.386535,11.995131,68.017101,34.187542,29.469475
min,2009-01-01 00:00:00,19.33000,NaN,9.140000,48.500000,1.210000,1.995000,8.500000,4.150000,3.100000
25%,2013-02-20 12:00:00,59.48000,NaN,13.840000,72.000000,2.580000,6.200000,39.887500,18.600000,15.820000
50%,2017-04-04 00:00:00,75.15000,NaN,17.150000,93.750000,3.060000,9.365000,54.800000,25.350000,21.800000
75%,2021-05-19 12:00:00,97.11000,NaN,22.270000,120.850000,3.970000,13.495000,69.525000,34.250000,27.437500
max,2025-07-07 00:00:00,127.98000,NaN,82.690000,457.800000,23.860000,69.955000,570.000000,310.000000,330.000000
std,NaN,23.53985,NaN,7.739013,74.698211,1.333616,9.385830,55.742973,32.768922,30.719771


Defining a plotting function

In [ ]:
import plotly.graph_objects as go
import pandas as pd

def plot_lstm_forecast_aligned_to_forecast_date(
    df_full,
    forecast_dates,
    y_pred_inv,
    y_true_inv,
    horizon_days,
    title='TTF Price Forecast (Aligned to Forecast Date)'
):
    """
    Plots historical TTF prices, and shows predicted & actual target prices on the forecast date.

    Parameters:
    - df_full: DataFrame with 'TTF_Price' and datetime index (historical)
    - forecast_dates: dates when forecasts were made
    - y_pred_inv: predicted prices for target dates (already inverse-transformed)
    - y_true_inv: actual target prices (already inverse-transformed)
    - horizon_days: how far ahead the forecast is (e.g., 30 for 1-month)
    - title: chart title
    """

    # Convert inputs to pandas Series for indexing
    forecast_dates = pd.to_datetime(forecast_dates)
    y_pred_series = pd.Series(y_pred_inv.flatten(), index=forecast_dates)

    # Shift actual target values back to the forecast date
    actual_target_dates = forecast_dates + pd.Timedelta(days=horizon_days)
    y_true_series = pd.Series(y_true_inv.flatten(), index=actual_target_dates)
    y_true_aligned = y_true_series.shift(-horizon_days, freq='D')  # move back to forecast date

    # Start building the plot
    fig = go.Figure()

    # Historical TTF price
    fig.add_trace(go.Scatter(
        x=df_full.index,
        y=df_full['TTF_Price'],
        mode='lines',
        name='Historical TTF Price',
        line=dict(color='lightgray'),
        opacity=0.5
    ))

    # Predicted target prices (plotted on forecast date)
    fig.add_trace(go.Scatter(
        x=y_pred_series.index,
        y=y_pred_series.values,
        mode='lines+markers',
        name=f'Predicted TTF (+{horizon_days}D)',
        marker=dict(size=2.5),
        line=dict(color='orange')
    ))

    # Actual future prices aligned to forecast date
    fig.add_trace(go.Scatter(
        x=y_true_aligned.index,
        y=y_true_aligned.values,
        mode='lines+markers',
        name=f'Actual TTF (+{horizon_days}D)',
        marker=dict(size=2.5),
        line=dict(color='blue')
    ))

    # Layout
    fig.update_layout(
        title=f'LSTM Forecast vs Actual TTF  ({horizon_days}-Day Horizon, Aligned to Forecast Date)',
        xaxis_title='Forecast Date',
        yaxis_title='TTF Price (EUR/MWh)',
        hovermode='x unified',
        xaxis=dict(tickangle=-45),
        height=600
    )

    fig.show()

In [ ]:
def forecast_metrics(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)

    return {
        "MAE": mae,
        "MSE": mse,
        "RMSE": rmse,
        "R²": r2
    }

In [ ]:
df_ttf = df_all[['Date', 'TTF_Price']].dropna()
df_ttf.set_index('Date', inplace=True) # Sets 'Date' as the DataFrame index so it behaves like a time series.
df_ttf = df_ttf.asfreq('D').interpolate() # Fills any gaps (e.g., holidays or weekends) using linear interpolation

## Tips for LSTM Success

| Tip                    | Why                                                          |
| ---------------------- | ------------------------------------------------------------ |
| Normalize data         | LSTMs perform better when values are between 0 and 1         |
| Use more lags          | Try 60, 90, or 180-day windows                               |
| Tune architecture      | Try stacked LSTMs, dropout, or learning rate schedules       |
| Add exogenous features | Include Brent, Coal, NBP, etc., as additional input channels |


In [ ]:
# Normalize the data
scaler = MinMaxScaler()
scaled = scaler.fit_transform(df_ttf[['TTF_Price']]) # Scales all prices to between 0 and 1 using Min-Max scaling, This helps LSTM learn better and converge faster.

# Create supervised Sequences

def create_sequences(data, window_size=30, horizon=30): # window_size=30: Use last 30 days of data as input/horizon=30: Predict the price 30 day into the future
    X, y = [], []
    for i in range(len(data) - window_size - horizon + 1):
        X.append(data[i:i+window_size])
        y.append(data[i+window_size+horizon-1])
    return np.array(X), np.array(y)

use 30 past days to predict 30 day ahead

In [ ]:
window_size = 90
horizon_days = 30
X, y = create_sequences(scaled, window_size=window_size, horizon=horizon_days) # Generates many (X, y) pairs for the entire dataset.

# Train/test split
split = int(len(X) * 0.8)
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

In [ ]:
y_test.shape

(1125, 1)

In [ ]:
# Build and Train the LSTM Model

# Build model
model_base = Sequential([
    LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dense(1)
])
model_base.compile(optimizer='adam', loss='mse',metrics=[
        MeanAbsoluteError(name='mae'),
        RootMeanSquaredError(name='rmse'),
        MeanAbsolutePercentageError(name='mape')
    ])

# Train model
history = model_base.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.1)

Epoch 1/20
133/133 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - loss: 4.4838e-04 - mae: 0.0151 - mape: 9140.2646 - rmse: 0.0199 - val_loss: 0.0056 - val_mae: 0.0449 - val_mape: 24.5383 - val_rmse: 0.0745
Epoch 2/20
133/133 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 6.9313e-05 - mae: 0.0060 - mape: 1036.0405 - rmse: 0.0083 - val_loss: 0.0059 - val_mae: 0.0455 - val_mape: 24.4881 - val_rmse: 0.0770
Epoch 3/20
133/133 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 6.5814e-05 - mae: 0.0060 - mape: 178.5575 - rmse: 0.0081 - val_loss: 0.0055 - val_mae: 0.0431 - val_mape: 22.6910 - val_rmse: 0.0741
Epoch 4/20
133/133 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 6.8896e-05 - mae: 0.0056 - mape: 1782.1958 - rmse: 0.0083 - val_loss: 0.0063 - val_mae: 0.0464 - val_mape: 24.8369 - val_rmse: 0.0791
Epoch 5/20
133/133 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 8.1898e-05 - mae: 0.0059 - mape: 4679.5322 - rmse: 0.0089 - val_loss: 0.0063 - val_mae: 0.0474 - val_mape: 26.2194 - val_rmse: 0.0796
Epoch 6/20
133/133 ━━━━━━━━━━

In [ ]:
# Evaluate and Inverse Scale Predictions
# Predict
y_pred = model_base.predict(X_test)

# Inverse scale
y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))
y_pred_inv = scaler.inverse_transform(y_pred)

37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step


In [ ]:
forecast_metrics(y_test_inv, y_pred_inv)

{'MAE': 17.11815686765768,
 'MSE': 1225.1863253028885,
 'RMSE': np.float64(35.00266168883287),
 'R²': 0.48791923180870667}

In [ ]:
# Evaluate and Inverse Scale Predictions
# Predict

# Calculate forecast dates based on the test set
# The first forecast date corresponds to the end of the first window in the test set
# The last forecast date corresponds to the end of the last window in the test set
forecast_dates = df_ttf.index[window_size - 1 : -(horizon_days)]
forecast_dates = forecast_dates[split:]

plot_lstm_forecast_aligned_to_forecast_date(
    df_full=df_ttf,
    forecast_dates=forecast_dates,
    y_pred_inv=y_pred_inv,
    y_true_inv=y_test_inv,
    horizon_days=30
)

use 90 days to predict 90 days ahead

In [ ]:
window_size = 365
horizon_days = 90
X, y = create_sequences(scaled, window_size=window_size, horizon=horizon_days) # Generates many (X, y) pairs for the entire dataset.

# Train/test split
split = int(len(X) * 0.8)
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# Build and Train the LSTM Model

# Build model
model = Sequential([
    LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dense(1)
])
model.compile(optimizer='adam', loss='mse')

# Train model
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.1)

Epoch 1/20


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



126/126 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - loss: 4.3242e-04 - val_loss: 0.0122
Epoch 2/20
126/126 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - loss: 1.2808e-04 - val_loss: 0.0123
Epoch 3/20
126/126 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - loss: 1.3021e-04 - val_loss: 0.0114
Epoch 4/20
126/126 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - loss: 1.2064e-04 - val_loss: 0.0092
Epoch 5/20
126/126 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - loss: 1.9728e-04 - val_loss: 0.0120
Epoch 6/20
126/126 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - loss: 1.2121e-04 - val_loss: 0.0124
Epoch 7/20
126/126 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - loss: 1.1665e-04 - val_loss: 0.0113
Epoch 8/20
126/126 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - loss: 1.1525e-04 - val_loss: 0.0108
Epoch 9/20
126/126 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - loss: 1.2021e-04 - val_loss: 0.0117
Epoch 10/20
126/126 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - loss: 1.1466e-04 - val_loss: 0.0118
Epoch 11/20
126/126 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - loss: 1.2172e-04 - val_loss: 0.0114
Epoch 12

In [ ]:
# Evaluate and Inverse Scale Predictions
# Predict
y_pred = model.predict(X_test)

# Inverse scale
y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))
y_pred_inv = scaler.inverse_transform(y_pred)

forecast_metrics(y_test_inv, y_pred_inv)

35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step


{'MAE': 25.14174479331182,
 'MSE': 1987.2337487631762,
 'RMSE': np.float64(44.57840002471125),
 'R²': 0.1897730075260815}

In [ ]:
# Evaluate and Inverse Scale Predictions
# Predict
y_pred = model.predict(X_test)

# Inverse scale
y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))
y_pred_inv = scaler.inverse_transform(y_pred)

# Calculate forecast dates based on the test set
# The first forecast date corresponds to the end of the first window in the test set
# The last forecast date corresponds to the end of the last window in the test set
forecast_dates = df_ttf.index[window_size - 1 : -(horizon_days)]
forecast_dates = forecast_dates[split:]

plot_lstm_forecast_aligned_to_forecast_date(
    df_full=df_ttf,
    forecast_dates=forecast_dates,
    y_pred_inv=y_pred_inv,
    y_true_inv=y_test_inv,
    horizon_days=horizon_days)

37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step


## Trying to add sin_day and cos_day feature on top of the TTF price or adding layers to LSTM model to see if will increase the performance....

In [7]:
# @title Filling missing values

def fill_missing_dates(df, date_column='date'):
    df[date_column] = pd.to_datetime(df[date_column])
    df = df.set_index(date_column).sort_index()

    # Reindex to continuous daily frequency
    full_range = pd.date_range(start=df.index.min(), end=df.index.max(), freq='D')
    df = df.reindex(full_range)

    # Optionally interpolate missing values
    df.interpolate(method='linear', inplace=True)

    # Reset index and rename
    df = df.reset_index().rename(columns={'index': date_column})
    return df

In [ ]:
df = df_all[['Date', 'TTF_Price']].dropna()
df = fill_missing_dates(df_ttf, date_column='Date')


In [ ]:
df['Date'] = pd.to_datetime(df['Date'])
df['dayofyear'] = df['Date'].dt.dayofyear
df['sin_day'] = np.sin(2 * np.pi * df['dayofyear'] / 365) # Smooth annual cycles,Ideal for yearly seasonality
df['cos_day'] = np.cos(2 * np.pi * df['dayofyear'] / 365) # sin_day and cos_day Encode Full Cycles, like latitude and longitude
df.head()

,Date,TTF_Price,dayofyear,sin_day,cos_day
0,2009-01-01,21.000000,1,0.017213,0.999852
1,2009-01-02,22.200000,2,0.034422,0.999407
2,2009-01-03,22.416667,3,0.051620,0.998667
3,2009-01-04,22.633333,4,0.068802,0.997630
4,2009-01-05,22.850000,5,0.085965,0.996298


In [ ]:
def create_sequences_multifeature(data, window_size=30, horizon=30):
    X, y = [], []
    for i in range(len(data) - window_size - horizon + 1):
        X.append(data[i:i+window_size])
        y.append(data[i+window_size+horizon-1][0])  # <== ONLY take the first feature (TTF_Price)
    return np.array(X), np.array(y)

In [ ]:
# Normalize the data

features = df[['TTF_Price', 'sin_day', 'cos_day']]
scaler = MinMaxScaler()
scaled = scaler.fit_transform(features)

window_size = 90
horizon_days = 30
test_split=0.2

X, y = create_sequences_multifeature(scaled, window_size=window_size, horizon=horizon_days)
split = int(len(X) * (1 - test_split))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]
print(f'X_train shape: {X_train.shape}')
print(f'X_test shape: {X_test.shape}')
print(f'y_train shape: {y_train.shape}')
print(f'y_test shape: {y_test.shape}')

X_train shape: (4716, 90, 3)
X_test shape: (1180, 90, 3)
y_train shape: (4716,)
y_test shape: (1180,)


In [ ]:
# Build and Train the LSTM Model

# Build model
model_3_features = Sequential([
    LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dense(1)
])
model_3_features.compile(optimizer=Adam(learning_rate=0.0005),
                         loss='mse')

# Train model
history = model_3_features.fit(X_train, y_train, epochs=50, callbacks=[EarlyStopping(patience=5)], batch_size=32, validation_split=0.1)



Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



133/133 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 0.1162 - val_loss: 0.0919
Epoch 2/50
133/133 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 0.1003 - val_loss: 0.0919
Epoch 3/50
133/133 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0991 - val_loss: 0.0918
Epoch 4/50
133/133 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 0.0992 - val_loss: 0.0918
Epoch 5/50
133/133 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.1001 - val_loss: 0.0917
Epoch 6/50
133/133 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0984 - val_loss: 0.0917
Epoch 7/50
133/133 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0996 - val_loss: 0.0917
Epoch 8/50
133/133 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0977 - val_loss: 0.0917
Epoch 9/50
133/133 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0975 - val_loss: 0.0916
Epoch 10/50
133/133 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.1008 - val_loss: 0.0916
Epoch 11/50
133/133 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0996 - val_loss: 0.0916
Epoch 12/50
133/133 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss:

In [ ]:
y_pred = model_3_features.predict(X_test).flatten()
y_pred.shape

37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step


(1180,)

In [ ]:
## Evaluate the model
y_pred = model_3_features.predict(X_test).flatten()

# Inverse scale: y_test and y_pred go back to original TTF_Price
y_test_inv = scaler.inverse_transform(
    np.hstack([y_test.reshape(-1, 1), np.zeros((len(y_test), 2))])
)[:, 0]

y_pred_inv = scaler.inverse_transform(
    np.hstack([y_pred.reshape(-1, 1), np.zeros((len(y_pred), 2))])
)[:, 0]

mse = mean_squared_error(y_test_inv, y_pred_inv)
mae = mean_absolute_error(y_test_inv, y_pred_inv)
rmse = np.sqrt(mse)


forecast_metrics(y_test_inv, y_pred_inv)

37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step


{'MAE': 88.33323507304198,
 'MSE': 10489.265683176445,
 'RMSE': np.float64(102.41711616315139),
 'R²': -3.3841096801955155}

In [ ]:
from tensorflow.keras.metrics import MeanAbsoluteError, RootMeanSquaredError, MeanAbsolutePercentageError
from tensorflow.keras.optimizers import Adam

model_3_features = Sequential([
    LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True),
    Dropout(0.2),
    LSTM(64, return_sequences=False),
    Dense(1)
])

model_3_features.compile(
    optimizer=Adam(learning_rate=0.0005),
    loss='mse',
    metrics=[
        MeanAbsoluteError(name='mae'),
        RootMeanSquaredError(name='rmse'),
        MeanAbsolutePercentageError(name='mape')
    ]
)

# Train model with EarlyStopping
history = model_3_features.fit(
    X_train, y_train,
    epochs=50,
    batch_size=32,
    validation_split=0.1,
    callbacks=[EarlyStopping(patience=5)],
    verbose=1
)

Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



126/126 ━━━━━━━━━━━━━━━━━━━━ 6s 31ms/step - loss: 4.4882e-04 - mae: 0.0154 - mape: 1226.2416 - rmse: 0.0201 - val_loss: 0.0118 - val_mae: 0.0797 - val_mape: 36.5270 - val_rmse: 0.1088
Epoch 2/50
126/126 ━━━━━━━━━━━━━━━━━━━━ 4s 30ms/step - loss: 1.3034e-04 - mae: 0.0089 - mape: 12747.6729 - rmse: 0.0114 - val_loss: 0.0121 - val_mae: 0.0809 - val_mape: 36.3386 - val_rmse: 0.1100
Epoch 3/50
126/126 ━━━━━━━━━━━━━━━━━━━━ 5s 27ms/step - loss: 1.2683e-04 - mae: 0.0089 - mape: 2965.6589 - rmse: 0.0113 - val_loss: 0.0115 - val_mae: 0.0785 - val_mape: 36.4855 - val_rmse: 0.1071
Epoch 4/50
126/126 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms/step - loss: 1.2157e-04 - mae: 0.0089 - mape: 705.4146 - rmse: 0.0110 - val_loss: 0.0106 - val_mae: 0.0739 - val_mape: 33.6218 - val_rmse: 0.1030
Epoch 5/50
126/126 ━━━━━━━━━━━━━━━━━━━━ 6s 35ms/step - loss: 1.2409e-04 - mae: 0.0091 - mape: 8354.4854 - rmse: 0.0111 - val_loss: 0.0126 - val_mae: 0.0844 - val_mape: 39.2331 - val_rmse: 0.1123
Epoch 6/50
126/126 ━━━━━━━━━━━━━━━━━

In [ ]:
## Evaluate the model
y_pred = model_3_features.predict(X_test).flatten()

# Inverse scale: y_test and y_pred go back to original TTF_Price
y_test_inv = scaler.inverse_transform(
    np.hstack([y_test.reshape(-1, 1), np.zeros((len(y_test), 2))])
)[:, 0]

y_pred_inv = scaler.inverse_transform(
    np.hstack([y_pred.reshape(-1, 1), np.zeros((len(y_pred), 2))])
)[:, 0]

mse = mean_squared_error(y_test_inv, y_pred_inv)
mae = mean_absolute_error(y_test_inv, y_pred_inv)
rmse = np.sqrt(mse)


forecast_metrics(y_test_inv, y_pred_inv)

35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step


{'MAE': 25.808492264481032,
 'MSE': 2037.015877077372,
 'RMSE': np.float64(45.13331227682466),
 'R²': 0.16947603736438555}

## trying to add other features to the LSTM model

In [8]:
df_other_features = df_all[['Date', 'TTF_Price', 'NBP_Price','Coal_Price', 'HenryHub_Price']].dropna()
df_other_features = fill_missing_dates(df_other_features, date_column='Date')
df_other_features['Date'] = pd.to_datetime(df_other_features['Date'])
df_other_features.set_index('Date', inplace=True)
df_other_features.head()

,TTF_Price,NBP_Price,Coal_Price,HenryHub_Price
Date,,,,
2009-08-03,8.85,20.25,70.63,3.43
2009-08-04,9.45,22.90,73.75,3.53
2009-08-05,9.25,22.50,73.65,3.61
2009-08-06,9.25,23.55,74.25,3.78
2009-08-07,9.23,23.25,74.05,3.57


In [9]:
df_other_features.describe()

,TTF_Price,NBP_Price,Coal_Price,HenryHub_Price
count,5801.000000,5801.000000,5801.000000,5801.000000
mean,30.115352,68.039089,114.310134,3.366995
std,30.945594,55.259784,75.461088,1.341175
min,3.100000,8.500000,48.750000,1.210000
25%,16.440000,40.000000,73.200000,2.550000
50%,22.100000,55.016667,94.600000,3.006667
75%,27.980000,69.916667,121.500000,3.960000
max,330.000000,570.000000,450.250000,23.860000


In [ ]:
# Normalize the data

features = df_other_features[['TTF_Price', 'NBP_Price','Coal_Price', 'HenryHub_Price']]
scaler = MinMaxScaler()
scaled = scaler.fit_transform(features)

window_size = 90
horizon_days = 30
test_split=0.2

X, y = create_sequences_multifeature(scaled, window_size=window_size, horizon=horizon_days)
split = int(len(X) * (1 - test_split))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]
print(f'X_train shape: {X_train.shape}')
print(f'X_test shape: {X_test.shape}')
print(f'y_train shape: {y_train.shape}')
print(f'y_test shape: {y_test.shape}')

X_train shape: (4545, 90, 4)
X_test shape: (1137, 90, 4)
y_train shape: (4545,)
y_test shape: (1137,)


In [ ]:
from tensorflow.keras.metrics import MeanAbsoluteError, RootMeanSquaredError, MeanAbsolutePercentageError
from tensorflow.keras.optimizers import Adam

model_other_features = Sequential([
    Input(shape=(X_train.shape[1], X_train.shape[2])),  # Preferred input shape
    LSTM(64, return_sequences=True),
    Dropout(0.2),
    LSTM(64),
    Dense(1)
])

model_other_features.compile(
    optimizer=Adam(),
    loss='mse',
    metrics=[
        MeanAbsoluteError(name='mae'),
        RootMeanSquaredError(name='rmse'),
        MeanAbsolutePercentageError(name='mape')
    ]
)

# Train model with EarlyStopping
history = model_other_features.fit(
    X_train, y_train,
    epochs=50,
    batch_size=32,
    validation_split=0.1,
    callbacks=[EarlyStopping(patience=5)],
    verbose=1
)

Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



128/128 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - loss: 1.9180e-04 - mae: 0.0100 - mape: 1958.1541 - rmse: 0.0132 - val_loss: 0.0070 - val_mae: 0.0513 - val_mape: 24.2942 - val_rmse: 0.0840
Epoch 2/50
128/128 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - loss: 6.9263e-05 - mae: 0.0064 - mape: 419.1206 - rmse: 0.0083 - val_loss: 0.0068 - val_mae: 0.0507 - val_mape: 24.3516 - val_rmse: 0.0827
Epoch 3/50
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 6.9919e-05 - mae: 0.0062 - mape: 6434.1777 - rmse: 0.0083 - val_loss: 0.0081 - val_mae: 0.0566 - val_mape: 26.9663 - val_rmse: 0.0899
Epoch 4/50
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 8.6373e-05 - mae: 0.0064 - mape: 4383.3228 - rmse: 0.0092 - val_loss: 0.0074 - val_mae: 0.0520 - val_mape: 24.3943 - val_rmse: 0.0858
Epoch 5/50
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 6.7815e-05 - mae: 0.0060 - mape: 6198.8853 - rmse: 0.0082 - val_loss: 0.0071 - val_mae: 0.0497 - val_mape: 22.8885 - val_rmse: 0.0840
Epoch 6/50
128/128 ━━━━━━━━━━━━━━━━━━

In [ ]:
## Evaluate the model
y_pred = model_other_features.predict(X_test).flatten()

# 2. Inverse scale both y_test and y_pred (only the TTF_Price column)
y_test_inv = scaler.inverse_transform(
    np.hstack([y_test.reshape(-1, 1), np.zeros((len(y_test), scaled.shape[1] - 1))])
)[:, 0]

y_pred_inv = scaler.inverse_transform(
    np.hstack([y_pred.reshape(-1, 1), np.zeros((len(y_pred), scaled.shape[1] - 1))])
)[:, 0]

forecast_metrics(y_test_inv, y_pred_inv)

36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step


{'MAE': 18.825039787378255,
 'MSE': 1439.4177450928105,
 'RMSE': np.float64(37.93965926432143),
 'R²': 0.4044353911311346}

In [ ]:
y_pred_inv

array([73.12252232, 72.69573791, 72.55842882, ..., 30.23851273,
       30.46255336, 30.66853161])

In [ ]:
# Calculate forecast dates based on the test set
# The first forecast date corresponds to the end of the first window in the test set
# The last forecast date corresponds to the end of the last window in the test set
forecast_dates = df_other_features.index[window_size - 1 : -(horizon_days)]
forecast_dates = forecast_dates[split:]

plot_lstm_forecast_aligned_to_forecast_date(
    df_full=df_other_features,
    forecast_dates=forecast_dates,
    y_pred_inv=y_pred_inv,
    y_true_inv=y_test_inv,
    horizon_days=horizon_days)

In [ ]:
# Try only TTF with coal_price

In [ ]:
# Normalize the data

features = df_other_features[['TTF_Price', 'Coal_Price']]
scaler = MinMaxScaler()
scaled = scaler.fit_transform(features)

window_size = 90
horizon_days = 30
test_split=0.2

X, y = create_sequences_multifeature(scaled, window_size=window_size, horizon=horizon_days)
split = int(len(X) * (1 - test_split))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]
print(f'X_train shape: {X_train.shape}')
print(f'X_test shape: {X_test.shape}')
print(f'y_train shape: {y_train.shape}')
print(f'y_test shape: {y_test.shape}')

X_train shape: (4545, 90, 2)
X_test shape: (1137, 90, 2)
y_train shape: (4545,)
y_test shape: (1137,)


In [ ]:
from tensorflow.keras.metrics import MeanAbsoluteError, RootMeanSquaredError, MeanAbsolutePercentageError
from tensorflow.keras.optimizers import Adam

model_ttf_coal = Sequential([
    Input(shape=(X_train.shape[1], X_train.shape[2])),  # Preferred input shape
    LSTM(64, return_sequences=True),
    Dropout(0.2),
    LSTM(64),
    Dense(1)
])

model_ttf_coal.compile(
    optimizer=Adam(),
    loss='mse',
    metrics=[
        MeanAbsoluteError(name='mae'),
        RootMeanSquaredError(name='rmse'),
        MeanAbsolutePercentageError(name='mape')
    ]
)

# Train model with EarlyStopping
history = model_ttf_coal.fit(
    X_train, y_train,
    epochs=50,
    batch_size=32,
    validation_split=0.1,
    callbacks=[EarlyStopping(patience=5)],
    verbose=1
)

Epoch 1/50
128/128 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - loss: 3.3480e-04 - mae: 0.0130 - mape: 1526.7937 - rmse: 0.0173 - val_loss: 0.0075 - val_mae: 0.0542 - val_mape: 25.8006 - val_rmse: 0.0867
Epoch 2/50
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 6.8126e-05 - mae: 0.0062 - mape: 3865.7126 - rmse: 0.0082 - val_loss: 0.0062 - val_mae: 0.0474 - val_mape: 21.8146 - val_rmse: 0.0788
Epoch 3/50
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 6.4605e-05 - mae: 0.0061 - mape: 1106.1039 - rmse: 0.0080 - val_loss: 0.0069 - val_mae: 0.0503 - val_mape: 23.4984 - val_rmse: 0.0830
Epoch 4/50
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - loss: 6.6901e-05 - mae: 0.0060 - mape: 5020.0249 - rmse: 0.0082 - val_loss: 0.0071 - val_mae: 0.0514 - val_mape: 24.1171 - val_rmse: 0.0841
Epoch 5/50
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - loss: 7.2775e-05 - mae: 0.0061 - mape: 433.2006 - rmse: 0.0085 - val_loss: 0.0065 - val_mae: 0.0490 - val_mape: 22.9403 - val_rmse: 0.0809
Epoch 6/50
128/128 ━━━━━━━

In [ ]:
## Evaluate the model
y_pred = model_ttf_coal.predict(X_test).flatten()

# 2. Inverse scale both y_test and y_pred (only the TTF_Price column)
y_test_inv = scaler.inverse_transform(
    np.hstack([y_test.reshape(-1, 1), np.zeros((len(y_test), scaled.shape[1] - 1))])
)[:, 0]

y_pred_inv = scaler.inverse_transform(
    np.hstack([y_pred.reshape(-1, 1), np.zeros((len(y_pred), scaled.shape[1] - 1))])
)[:, 0]

forecast_metrics(y_test_inv, y_pred_inv)

36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step


{'MAE': 16.821991654129082,
 'MSE': 1143.2611863356399,
 'RMSE': np.float64(33.81214554469503),
 'R²': 0.5269713023921083}

Try to fine tune the LSTM hyper parameters

In [ ]:
param_grid = {
    'window_sizes': [30, 90, 180],
    'dropouts': [0.2, 0.4, 0.5],
    'learning_rates': [0.001, 0.0001]
}


results = []

for window_size in param_grid['window_sizes']:
    for dropout in param_grid['dropouts']:
        for lr in param_grid['learning_rates']:

            # Prepare sequences
            X, y = create_sequences_multifeature(scaled, window_size=window_size, horizon=horizon_days)
            split = int(len(X) * (1 - test_split))
            X_train, X_test = X[:split], X[split:]
            y_train, y_test = y[:split], y[split:]

            # Build model
            model = Sequential([
                Input(shape=(X_train.shape[1], X_train.shape[2])),
                LSTM(64, return_sequences=True),
                Dropout(dropout),
                LSTM(64),
                Dense(1)
            ])
            model.compile(
                optimizer=Adam(learning_rate=lr),
                loss='mse',
                metrics=['mae']
            )

            # Train
            history = model.fit(
                X_train, y_train,
                epochs=50,
                batch_size=32,
                validation_split=0.1,
                callbacks=[EarlyStopping(patience=5)],
                verbose=0
            )

            # Predict
            y_pred = model.predict(X_test).flatten()

            # Inverse scale
            y_test_inv = scaler.inverse_transform(
                np.hstack([y_test.reshape(-1, 1), np.zeros((len(y_test), scaled.shape[1] - 1))])
            )[:, 0]

            y_pred_inv = scaler.inverse_transform(
                np.hstack([y_pred.reshape(-1, 1), np.zeros((len(y_pred), scaled.shape[1] - 1))])
            )[:, 0]

            # Metrics
            mae = mean_absolute_error(y_test_inv, y_pred_inv)
            rmse = np.sqrt(mean_squared_error(y_test_inv, y_pred_inv))
            r2 = r2_score(y_test_inv, y_pred_inv)

            results.append({
                'window_size': window_size,
                'dropout': dropout,
                'learning_rate': lr,
                'mae': mae,
                'rmse': rmse,
                'r2': r2
            })

            print(f"✅ Tested window={window_size}, dropout={dropout}, lr={lr} ➝ MAE={mae:.4f}, RMSE={rmse:.4f}, r2 = {r2}")


36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
✅ Tested window=30, dropout=0.2, lr=0.001 ➝ MAE=16.4730, RMSE=33.1861, r2 = 0.5420156559940155
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
✅ Tested window=30, dropout=0.2, lr=0.0001 ➝ MAE=17.7135, RMSE=35.0804, r2 = 0.48823867542710186
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
✅ Tested window=30, dropout=0.4, lr=0.001 ➝ MAE=16.4826, RMSE=32.8751, r2 = 0.5505579911517133
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step
✅ Tested window=30, dropout=0.4, lr=0.0001 ➝ MAE=18.1537, RMSE=35.0205, r2 = 0.4899843277511956
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step
✅ Tested window=30, dropout=0.5, lr=0.001 ➝ MAE=17.4415, RMSE=35.1105, r2 = 0.4873605648624839
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
✅ Tested window=30, dropout=0.5, lr=0.0001 ➝ MAE=17.4017, RMSE=34.4293, r2 = 0.5070582790967095
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
✅ Tested window=90, dropout=0.2, lr=0.001 ➝ MAE=16.9924, RMSE=33.4569, r2 = 0.5368582209791319
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
✅ Tested window=9